In [1]:
import newspaper #For finding today's trending topics
import datetime as dt 
from datetime import date
#from __future__ import print_function, unicode_literals
import spacy #For nlp prep
from collections import defaultdict, Counter
import shutil
import requests #For our REST API calls
import schedule
import time
import tweepy #To Post to Twitter
import numpy as np
nlp = spacy.load('en') #For NLP later

#References: 
#https://www.pluralsight.com/guides/interesting-apis/build-a-simpsons-quote-bot-with-twilio-mms-frinkiac-and-python
#


# Setup your Twitter Account Keys

In [2]:
consumer_key = 'XXXXXXXXXXXXXXXXXXXXXXXXX'
consumer_secret = 'YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY'
access_token = 'WWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWW'
access_token_secret = 'ZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZ'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# Find Today's Trending Topics

## Filter topics for nouns

In [3]:
#Poll Google trends to get hot noun trends with the newspaper package

def get_hot_nouns():
    pos_counts = defaultdict(Counter)
    
    nouns=[]
    hots = newspaper.hot() #Get today's trending topics
    hots = [' {0}'.format(elem) for elem in hots] #add a space to each one
    hots = ''.join(hots) #Join them into one big string
    
    doc = nlp(hots)
    
    doc = nlp(hots) #Create Spacy Doc
    
    for token in doc:
        pos_counts[token.pos][token.orth] += 1

    for pos_id, counts in sorted(pos_counts.items()):
        pos = doc.vocab.strings[pos_id]
        for orth_id, count in counts.most_common():
            #print(pos, count, doc.vocab.strings[orth_id])
            if pos == 'NOUN':
                nouns.append(doc.vocab.strings[orth_id])
                
    nouns = ['{0} '.format(elem) for elem in nouns] #add a space to each one
    nouns = ''.join(nouns) #Join them into one big string
    return(nouns)

## Create Hashtags for Twitter Post

In [4]:
#Poll Google trends to get hot noun trends

def get_hot_nouns_hashtagged():
    pos_counts = defaultdict(Counter)
    
    nouns=[]
    hots = newspaper.hot() #Get today's trending topics
    hots = [' {0}'.format(elem) for elem in hots] #add a space to each one
    hots = ''.join(hots) #Join them into one big string
    
    doc = nlp(hots)
    
    doc = nlp(hots) #Create Spacy Doc
    
    for token in doc:
        pos_counts[token.pos][token.orth] += 1

    for pos_id, counts in sorted(pos_counts.items()):
        pos = doc.vocab.strings[pos_id]
        for orth_id, count in counts.most_common():
            #print(pos, count, doc.vocab.strings[orth_id])
            if pos == 'NOUN':
                nouns.append(doc.vocab.strings[orth_id])
                
    nouns = ['#{0} '.format(elem) for elem in nouns] #add a space to each one
    nouns = ''.join(nouns) #Join them into one big string
    return(nouns)

In [5]:
get_hot_nouns_hashtagged()

'#Destiny #Release #Date #office '

# Create Your Twitter Caption w/Frinkiac API

In [ ]:
def get_meme_caption():
    hot_nouns = get_hot_nouns()
    url_image = "https://frinkiac.com/api/search?q=" + hot_nouns
    r = requests.get(url_image) 
    json = r.json() 
        # Extract the episode number and timestamp from the API response 
        # and convert them both to strings. 
    ID, episode, timestamp = map(str, json[0].values()) 
    image_url = "https://frinkiac.com/meme/" + episode + "/" + timestamp 
         #Combine each line of subtitles into one string. 
    #caption = "\n".join([subtitle["Content"] for subtitle in json["Subtitles"]]) 

    url_caption = "http://frinkiac.com/api/caption?e=" + episode +"&t=" + timestamp
    r = requests.get(url_caption) 
    json = r.json() 
    ID, episode, timestamp = map(str, json["Frame"].values())  
    image_url = "https://frinkiac.com/meme/" + episode + "/" + timestamp 
        # Combine each line of subtitles into one string. 
    caption = " ".join([subtitle["Content"] for subtitle in json["Subtitles"]])
    j=0
    hot_nouns_hashtag = []
    hot_noun_hashtag = []
    for hot_noun in hot_nouns:
        hot_noun_hashtag = ("#" + hot_nouns[j])
        hot_nouns_hashtag.append(hot_noun_hashtag)
        j=j+1
    
    hot_nouns_hashtagged = get_hot_nouns_hashtagged()
    caption = "Simpsons Bot Says: " + caption + hot_nouns_hashtagged
    response = requests.get(image_url, stream=True)
    with open('img.png', 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)
    del response
    return(caption)


# Send Tweet

In [ ]:
def send_tweet(): 
    caption = get_meme_caption() 
    api.update_with_media(filename='img.png', status=caption)
    print("Tweet sent!") # If an error occurs, print it out 

# Schedule Tweets

In [ ]:
#schedule it
schedule.every().day.at("10:00").do(send_tweet)

while True:
    schedule.run_pending()

# Create Script for Automation

In [ ]:
#1. Convert to Script:
#jupyter nbconvert --to script SimpsonsBot.ipynb

#2.Create .sh rile:
####
#'#!/bin/bash;
#'python /Users/whs/Desktop/StartUp/SimpsonsBot.py;
####

#3. Now make the .sh excecutable: 
#chmod a+x (yourscriptname.sh)

#4. Trigger .sh file to run on startup :
# MacOS: System Preferences >> Users and Groups >> Login Items
